In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pyproj import Transformer
import matplotlib.pyplot as plt
import seaborn as sns
import folium


In [2]:
crime_data = pd.read_csv('LSOA_code_crime_count.csv')
lsoa_map = gpd.read_file('./lsoa_shape/infuse_lsoa_lyr_2011.shp')

lsoa_map['geometry'] = lsoa_map['geometry'].simplify(tolerance=5, preserve_topology=True)


In [3]:
crime_data.head()

,LSOA code,Total population,Crime count,Crimes per 1000
0,E01011949,1866,272.0,145.766345
1,E01011950,1094,547.0,500.000000
2,E01011951,1240,565.0,455.645161
3,E01011952,1620,430.0,265.432099
4,E01011953,1979,320.0,161.697827


In [4]:
lsoa_map.head()

,geo_code,geo_label,geo_labelw,label,name,geometry
0,E01003513,Newham 035D,None,E92000001E09000025E01003513,Newham 035D,"POLYGON ((541893.189 181249.621, 541912.881 18..."
1,E01031647,Horsham 002D,None,E92000001E07000227E01031647,Horsham 002D,"POLYGON ((518376.682 132574.695, 518384.193 13..."
2,E01022006,Tendring 002C,None,E92000001E07000076E01022006,Tendring 002C,"POLYGON ((623754.716 231042.037, 623775.437 23..."
3,E01001159,Croydon 002C,None,E92000001E09000008E01001159,Croydon 002C,"POLYGON ((532233.977 170474.976, 532204.000 17..."
4,E01008088,Sheffield 012B,None,E92000001E08000019E01008088,Sheffield 012B,"POLYGON ((433539.233 392096.845, 433499.470 39..."


In [5]:
merged_geo = lsoa_map.merge(crime_data, left_on='geo_code', right_on='LSOA code', how='left')
merged_geo.head()

,geo_code,geo_label,geo_labelw,label,name,geometry,LSOA code,Total population,Crime count,Crimes per 1000
0,E01003513,Newham 035D,None,E92000001E09000025E01003513,Newham 035D,"POLYGON ((541893.189 181249.621, 541912.881 18...",E01003513,1546.0,373.0,241.267788
1,E01031647,Horsham 002D,None,E92000001E07000227E01031647,Horsham 002D,"POLYGON ((518376.682 132574.695, 518384.193 13...",E01031647,1206.0,48.0,39.800995
2,E01022006,Tendring 002C,None,E92000001E07000076E01022006,Tendring 002C,"POLYGON ((623754.716 231042.037, 623775.437 23...",E01022006,1805.0,260.0,144.044321
3,E01001159,Croydon 002C,None,E92000001E09000008E01001159,Croydon 002C,"POLYGON ((532233.977 170474.976, 532204.000 17...",E01001159,1963.0,159.0,80.998472
4,E01008088,Sheffield 012B,None,E92000001E08000019E01008088,Sheffield 012B,"POLYGON ((433539.233 392096.845, 433499.470 39...",E01008088,1281.0,131.0,102.263856


In [6]:
merged_geo.isna().sum()

geo_code                0
geo_label               0
geo_labelw          35643
label                   0
name                    0
geometry                0
LSOA code           10267
Total population    10267
Crime count         10267
Crimes per 1000     10267
dtype: int64

In [7]:
merged_geo = merged_geo.drop(columns=['geo_labelw'])
merged_geo = merged_geo.fillna(-1)


merged_geo.isnull().sum()


geo_code            0
geo_label           0
label               0
name                0
geometry            0
LSOA code           0
Total population    0
Crime count         0
Crimes per 1000     0
dtype: int64

In [8]:
def get_color(crime_rate):
    if crime_rate == -1:
        return 'cyan'
    if crime_rate <= 37.484700341790536:
        return 'green'
    elif crime_rate <= 57.38737955096794:
        return 'yellow'
    elif crime_rate <= 84.57123998114098:
        return 'orange'
    elif crime_rate <= 130.50818474077755:
        return 'red'
    else:
        return 'darkred'


In [9]:
center_lat = merged_geo.geometry.centroid.y.mean()
center_lon = merged_geo.geometry.centroid.x.mean()
m = folium.Map(location=[51.508454,0.045997], zoom_start=12)

folium.GeoJson(
    merged_geo,
    style_function=lambda x: {
        'fillColor': get_color(x['properties']['Crimes per 1000']),
        'color': 'black',  # Outline color
        'weight': 0.5,       # Outline weight
        'fillOpacity': 0.4
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['geo_code', 'Total population', 'Crime count', 'Crimes per 1000'],
        aliases=['LSOA Code:', 'Total Population:', 'Crime Count:', 'Crimes per 1000:'],
        localize=True
    )
).add_to(m)
m.save('LSOA_crime_map.html')


In [45]:
subset_merged_geo = merged_geo.iloc[:1000]
print(subset_merged_geo)
map = folium.Map(location=[51.508454,0.045997], zoom_start=12)
folium.GeoJson(
    subset_merged_geo,
    style_function=lambda x: {
        'fillColor': get_color(x['properties']['Crimes per 1000']),
        'color': 'black',  # Outline color
        'weight': 0.5,       # Outline weight
        'fillOpacity': 0.7
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=['geo_code', 'Total population', 'Crime count', 'Crimes per 1000'],
        aliases=['LSOA Code:', 'Total Population:', 'Crime Count:', 'Crimes per 1000:'],
    )
).add_to(map)
map.save('subset_map.html')


       geo_code                   geo_label                        label  \
0     E01003513                 Newham 035D  E92000001E09000025E01003513   
1     E01031647                Horsham 002D  E92000001E07000227E01031647   
2     E01022006               Tendring 002C  E92000001E07000076E01022006   
3     E01001159                Croydon 002C  E92000001E09000008E01001159   
4     E01008088              Sheffield 012B  E92000001E08000019E01008088   
...         ...                         ...                          ...   
1090  E01032486  Blackburn with Darwen 018F  E92000001E06000008E01032486   
1091  E01033327               Lewisham 012F  E92000001E09000023E01033327   
1092  E01009878                 Dudley 025E  E92000001E08000027E01009878   
1093  E01026139                Lincoln 011C  E92000001E07000138E01026139   
1094  W01001753                Cardiff 031C  W92000004W06000015W01001753   

                            name  \
0                    Newham 035D   
1              